In [36]:
#IMPORTS
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier

In [30]:
def load_data(df):
    y=df.quality
    X=df.drop('quality',1)
    return X, y

In [31]:
#CREATE AND MERGE DATAFRAMES
dfRed=pd.read_csv("winequality-red.csv", sep=";")
dfWhite=pd.read_csv("winequality-white.csv", sep=";")

#TODO: Perque no ens deixa posar red i white
dfRed['Type']="0" #Red  
dfWhite['Type']="1" #White
df=pd.concat([dfRed,dfWhite])
df.sample(frac=1)

X, y = load_data(df)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

C:\Users\walli\AppData\Local\Temp/ipykernel_1576/4001844796.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X=df.drop('quality',1)


In [32]:
#LOGISTIC REGRESSION

logreg = LogisticRegression()
logreg.fit(X_train,y_train)

y_predict = logreg.predict(X_test)

dfLogReg=pd.DataFrame()
dfLogReg['Real'] = y_test
dfLogReg['Prediction']=y_predict
print(dfLogReg.head)

training_accuracy = logreg.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = logreg.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

#TODO: Perquè dona aquells warnings?

<bound method NDFrame.head of       Real  Prediction
2173     5           5
4410     5           6
1865     6           6
1518     5           6
2162     6           5
...    ...         ...
2677     6           6
3683     6           6
2255     6           6
1759     5           5
4267     7           6

[1625 rows x 2 columns]>
training_accuracy: 0.47311165845648606
testing_accuracy: 0.4775384615384615


C:\Users\walli\miniconda3\envs\ml-environment\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [33]:
#PERCEPTRON
perceptron = Perceptron()
perceptron.fit(X_train, y_train)

y_predict=perceptron.predict(X_test)

dfPerceptron=pd.DataFrame()
dfPerceptron['Real'] = y_test
dfPerceptron['Prediction']=y_predict
print(dfPerceptron.head)

training_accuracy = perceptron.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = perceptron.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

<bound method NDFrame.head of       Real  Prediction
2173     5           6
4410     5           6
1865     6           6
1518     5           6
2162     6           6
...    ...         ...
2677     6           6
3683     6           6
2255     6           6
1759     5           6
4267     7           6

[1625 rows x 2 columns]>
training_accuracy: 0.4351395730706076
testing_accuracy: 0.43815384615384617


In [37]:
#RANDOM FOREST
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_predict=rf.predict(X_test)

dfRF=pd.DataFrame()
dfRF['Real'] = y_test
dfRF['Prediction']=y_predict
print(dfRF.head)

training_accuracy = rf.score(X_train,y_train)
print('training_accuracy:',training_accuracy)

testing_accuracy = rf.score(X_test,y_test)
print('testing_accuracy:',testing_accuracy)

<bound method NDFrame.head of       Real  Prediction
2173     5           5
4410     5           5
1865     6           5
1518     5           6
2162     6           5
...    ...         ...
2677     6           6
3683     6           6
2255     6           6
1759     5           5
4267     7           6

[1625 rows x 2 columns]>
training_accuracy: 1.0
testing_accuracy: 0.6750769230769231
